# Identifying Fake Bills

The Introduction goes here

In [5]:
library(tidyverse)  # for general tidyverse functions
library(tidymodels) # for making the training/testing split
library(ggplot2)    # for producing plots
library(repr)       # for adjusting plots

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

In [6]:
# reading and tidying the dataset
fake_bills <- read_csv("https://raw.githubusercontent.com/heeyachung/dsci-group-14-/main/fakebills.csv") |> #reading the file
    mutate(is_genuine = as_factor(is_genuine)) |> #factoring the legitimacy variable
    mutate(is_genuine = fct_recode(is_genuine, "real" = "1", "fake" = "2")) #renaming values

# splitting the data into the training and testing split
fake_bills_split <- initial_split(fake_bills, prop = 0.75, strata = is_genuine)
bills_training <- training(fake_bills_split)  # training split
bills_testing <- testing(fake_bills_split)    # testing split
head(bills_training)

Rows: 1500 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (7): is_genuine, diagonal, height_left, height_right, margin_low, margin...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fake,171.59,104.14,104.38,4.97,3.47,111.22
fake,172.55,104.25,104.23,5.60,3.13,111.72
fake,171.88,104.30,104.18,5.34,3.33,112.69
fake,171.63,104.05,104.25,4.61,3.10,110.91
fake,171.83,104.13,104.52,4.94,3.27,111.72
fake,172.30,104.28,103.90,5.10,3.57,110.66


In [7]:
bills_class <- bills_training |>
    group_by(is_genuine) |>
    summarize(count = n()) 
bills_class

is_genuine,count
<fct>,<int>
real,750
fake,375


As previously discussed in the initial analysis in our proposal, there is a large disparity between the number of real and fake bills. As said disparity would negatively affect the efficacy of our model, we will produce synthetic data points for fake bills.